## Preprocessing

- Check if all docs only contain ASCII letters

In [ ]:
prep_docs = []
for doc in docs:
    prep_doc = ''
    for sentence in doc:
        sentence = preprocess_text(sentence) # merge_punctuation(sentence)
        print(sentence)
        prep_doc = ' '.join([prep_doc, sentence])

    prep_docs.append(prep_doc)

# for idx, prep_doc in enumerate(prep_docs):
#     print(f'Doc {idx} is ACSII: {prep_doc.isascii()}')

[idx for idx, prep_doc in enumerate(prep_docs) if prep_doc.isalnum() == False]

# [print(prep_doc) for prep_doc in enumerate(prep_doc) if prep_doc.isascii() == False]

- Sort corpus vocab and dump to json

In [ ]:
corpus_vocab = dict(sorted(corpus_vocab.items()))

len(corpus_vocab)

print(corpus_vocab)

import json
with open('corpus_vocab.json', 'w') as f:
    json.dump(corpus_vocab, f)

## Working with BERT

In [16]:
from transformers import BertTokenizer, BertModel
BERT_preprocessing_tokenizer = "bert-base-uncased"
BERT_model = 'bert-base-uncased'

In [3]:
tokenizer = BertTokenizer.from_pretrained(BERT_preprocessing_tokenizer)

In [11]:
text = 'While and as well as or operator needs two operands, which may evaluate to true or false, not operator needs one operand evaluating to true or false.'
marked_text = "[CLS] " + text + " [SEP]"

In [26]:
tokenized_sentence = tokenizer.tokenize(marked_text)

len(tokenized_sentence)

34

In [28]:
# import torch

# def get_bert_embedding_input(sentence):
#     tokenized_sentence = tokenizer.tokenize(sentence)
#     index_tokens = tokenizer.convert_tokens_to_ids(tokenized_sentence)
#     tokens_tensor = torch.tensor([index_tokens])
#     segments_tensor = torch.ones(len(tokenized_sentence))

#     return tokenized_sentence, index_tokens, tokens_tensor, segments_tensor

In [125]:
class BertSentenceEmbedding:
    def __init__(self, sentence):
        self.sentence = sentence
        self.tokenized_sentence = tokenizer.tokenize(sentence)
        self.index_tokens = tokenizer.convert_tokens_to_ids(self.tokenized_sentence)
        self.tokens_tensor = torch.tensor([self.index_tokens])
        self.segments_tensor = torch.tensor([[1 for _ in self.tokenized_sentence]])

In [127]:
s = BertSentenceEmbedding(text)

In [123]:
s.segments_tensor
# s.index_tokens

tensor([[1, 1, 1]])

In [128]:
model.eval()

with torch.no_grad():
    outputs = model(s.tokens_tensor, s.segments_tensor)
    hidden_states = outputs[2]

In [137]:
import numpy as np

np.shape(hidden_states)

c:\Users\93cha\.conda\envs\thesis\lib\site-packages\numpy\core\fromnumeric.py:2009: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape
c:\Users\93cha\.conda\envs\thesis\lib\site-packages\numpy\core\fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(13,)

In [145]:
print(len(hidden_states)) # Number of layers in BERT models, including the inout embeddings
print(len(hidden_states[0])) # Number of batches (sentences) input
print(len(hidden_states[0][0])) # Number of tokens in sentence
print(len(hidden_states[0][0][0])) # Number of hidden units / features

13
1
20
768


In [148]:
hidden_states[0][0][0]

tensor([ 2.4174e-01, -2.3037e-01,  1.0960e-02,  1.4319e-01, -2.8272e-01,
         1.0180e-01,  5.5519e-01, -5.4578e-02, -2.2493e-01, -8.6961e-01,
         4.4585e-01,  1.3074e-01, -8.0233e-02, -2.3877e-01, -4.7602e-01,
         4.5088e-01,  2.8182e-01, -7.5689e-02, -4.3809e-01,  1.3318e-02,
        -4.8195e-01, -6.3182e-01,  2.1935e-01,  3.7190e-01,  2.3699e-01,
         1.7386e-01, -4.3828e-01,  1.8428e-01, -3.6722e-01,  4.6221e-01,
        -3.2307e-01, -5.2181e-02,  1.2818e-01, -1.8686e-02,  1.1297e-01,
         2.1369e-01, -6.5989e-01,  1.6764e-01, -1.2653e-02,  8.2431e-01,
        -7.1396e-02,  4.1277e-01, -2.8675e-01,  6.0078e-01, -1.8392e-01,
        -2.3079e-01, -3.5267e+00,  3.7276e-01,  5.7021e-01,  1.4452e-02,
         2.8992e-01,  3.2949e-01,  6.9618e-02,  8.4421e-01,  2.4687e-02,
        -4.9501e-01,  3.0071e-01, -2.1781e-01, -3.5601e-01, -2.6869e-01,
        -7.0847e-02,  7.6872e-01, -6.4508e-01, -1.7380e-01,  2.9230e-01,
        -1.3748e-01, -1.9912e-01, -4.6313e-01, -1.3

- Create / add token embeddings

In [ ]:
doc_token_embeddings = {}

tokens = ['i', 'love', 'you']

token_embeddings = get_token_embeddings(tokens)

# doc_token_embeddings = dict(zip(tokens, token_embeddings))

In [ ]:
tokens = ['i', 'like', 'him', 'and', 'you']

token_embeddings = get_token_embeddings(tokens)

print(token_embeddings)

tensor([[-0.1019, -0.8678, -0.1187,  ..., -0.0555,  0.3492,  0.7548],
        [ 0.7621, -0.1919, -0.1353,  ...,  0.0494,  0.4771,  0.4659],
        [ 0.4800, -0.1612, -0.7899,  ..., -0.5469,  0.8261,  0.3935],
        [ 0.4322, -0.5650, -0.4019,  ...,  0.0614,  0.2550,  0.3160],
        [ 0.0070, -0.0818, -0.2650,  ..., -0.2173,  0.7020,  0.5809]])


In [22]:
window = ['i', 'love', 'you']

In [36]:
window = ['i', 'love', 'you']

adj = {}

In [31]:
adj

{'love': 1, 'i': 1}

In [34]:
for token in window:
    if token not in adj.keys(): adj[token] = {}
    window_temp = window
    window_temp.remove(token)
    for token_temp in window_temp:
        if token_temp not in adj[token].keys():
            adj[token][token_temp] =1
        else: adj[token][token_temp] +=1

In [35]:
# window.remove('love')
adj

{'you': {}}